# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 4: Web Scraping Job Postings

## Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:

   1. Determine the industry factors that are most important in predicting the salary amounts for these data.
   2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. 

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer these two questions.

---

## Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to answer the two questions described above.

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

### QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:
- What components of a job posting distinguish data scientists from other data jobs?
- What features are important for distinguishing junior vs. senior positions?
- Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?

You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your models' performance.


### BONUS PROBLEM

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs. Plot the ROC curve.

---

## Requirements

1. Scrape and prepare your own data.

2. **Create and compare at least two models for each section**. One of the two models should be a decision tree or ensemble model. The other can be a classifier or regression of your choosing (e.g. Ridge, logistic regression, KNN, SVM, etc).
   - Section 1: Job Salary Trends
   - Section 2: Job Category Factors

3. Prepare a polished Jupyter Notebook with your analysis for a peer audience of data scientists. 
   - Make sure to clearly describe and label each section.
   - Comment on your code so that others could, in theory, replicate your work.

4. A brief writeup in an executive summary, written for a non-technical audience.
   - Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

#### BONUS

5. Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions.

6. Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

## Questions to Answer

### 1) Determine the `industry factors` that are most important in predicting the salary amounts for these data 

### 2) Determine the `factors that distinguish job categories and titles` from each other. For example, can required skills accurately predict job title?

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
pd.set_option('display.max_columns',None)

In [2]:
df=pd.read_csv('data.csv')

In [3]:
df.head(2)

,Unnamed: 0,company_name,job_address,job_title,job_category,employment_type,salary,salary_type,description,requirements
0,0,GOOGLE ASIA PACIFIC PTE. LTD.,"MARINA BAY FINANCIAL CENTRE, 8 MARINA BOULEVAR...","Data Science Lead, Large Customer Sales - Sing...",Information Technology,Full Time,"$7,700to$15,400",Monthly,Roles & ResponsibilitiesCompany overview: Goog...,RequirementsMinimum qualifications: - Bachelor...
1,1,TRAKOMATIC PTE. LTD.,NaN,Data analyst,Information Technology,Full Time,"$3,500to$4,500",Monthly,Roles & ResponsibilitiesThe Retail Data Analys...,RequirementsRequirements: A bachelor’s deg...


In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

## 1) EDA and Cleaning Salary

In [5]:
df.salary.isnull().sum()

210

In [6]:
# cleaning the salary column and splitting the information to maximum salary and minimum salary
df['salary_min']=df.salary.str.replace(',','').str.replace('to',' ').str.replace('$','').str.split().str[0].astype(float)
df['salary_max']=df.salary.str.replace(',','').str.replace('to',' ').str.replace('$','').str.split().str[1].astype(float)


In [7]:
# create a salary average column
df['salary_average']=(df['salary_min']+df['salary_max'])/2

In [8]:
df['salary_average'].describe()

count      1790.000000
mean       9271.484358
std       20936.753789
min         650.000000
25%        3750.000000
50%        6000.000000
75%        7750.000000
max      350000.000000
Name: salary_average, dtype: float64

In [9]:
# converting annual salary to monthly salary
for i in df[df['salary_type'].isin(['Annually'])].index:
    df['salary_average'][i]=df[df['salary_type'].isin(['Annually'])]['salary_average'][i]/12


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df['salary_average'].describe()

count      1790.000000
mean       6539.453073
std        7183.729083
min         650.000000
25%        3750.000000
50%        6000.000000
75%        7500.000000
max      180000.000000
Name: salary_average, dtype: float64

In [11]:
# investigating the job with the oddly high monthly pay
# Since the job is looking for a 'boss', the pay seems legit
df[df['salary_average']==180000]

,company_name,job_address,job_title,job_category,employment_type,salary,salary_type,description,requirements,salary_min,salary_max,salary_average
1600,SIX FINANCIAL INFORMATION SINGAPORE PTE. LTD.,"SUNTEC TOWER TWO, 9 TEMASEK BOULEVARD 038989",Head of SSO (APAC),"Banking and Finance, Others",Full Time,"$144,000to$216,000",Monthly,Roles & ResponsibilitiesThe SSO group provides...,RequirementsMain Duties 1. A key positive aspe...,144000.0,216000.0,180000.0


In [12]:
# dropping salary column
df.drop(columns='salary',inplace=True)

In [13]:
# dropping rows that doesnt have salary value
df.dropna(subset=['salary_average'],inplace=True)

In [14]:
# creating a target variable ( high pay=1, low pay=0)
df['salary_high_low']=df['salary_average'].map(lambda x: 0 if x<6000 else 1)

In [15]:
df.head()

,company_name,job_address,job_title,job_category,employment_type,salary_type,description,requirements,salary_min,salary_max,salary_average,salary_high_low
0,GOOGLE ASIA PACIFIC PTE. LTD.,"MARINA BAY FINANCIAL CENTRE, 8 MARINA BOULEVAR...","Data Science Lead, Large Customer Sales - Sing...",Information Technology,Full Time,Monthly,Roles & ResponsibilitiesCompany overview: Goog...,RequirementsMinimum qualifications: - Bachelor...,7700.0,15400.0,11550.0,1
1,TRAKOMATIC PTE. LTD.,NaN,Data analyst,Information Technology,Full Time,Monthly,Roles & ResponsibilitiesThe Retail Data Analys...,RequirementsRequirements: A bachelor’s deg...,3500.0,4500.0,4000.0,0
3,INTELLECT MINDS PTE. LTD.,"ANSON CENTRE, 51 ANSON ROAD 079904",Data Engineer,Information Technology,Full Time,Monthly,Roles & ResponsibilitiesCompany Overview Intel...,RequirementsQualifications • Advanced working ...,5000.0,7000.0,6000.0,1
5,GUMI ASIA PTE. LTD.,"CONNEXIS, 1 FUSIONOPOLIS WAY 138632",Data Analyst,Others,Full Time,Monthly,Roles & Responsibilities Perform evaluations a...,Requirements Minimum Diploma/ Degree in Statis...,2800.0,3800.0,3300.0,0
6,DBS BANK LTD.,"MARINA BAY FINANCIAL CENTRE, 12 MARINA BOULEVA...","AVP, Data Scientist, Business Analytics, Consu...",Information Technology,Full Time,Monthly,Roles & ResponsibilitiesBusiness Function As ...,Requirements Masters or equivalent degree in S...,6500.0,11700.0,9100.0,1


## EDA and Cleaning Employment Type

In [16]:
# selecting reliable employment type information

permanent_full_time=['Full Time', 'Permanent', 'Permanent, Full Time']
temporary_full_time=['Contract','Permanent, Contract','Permanent, Contract, Full Time','Contract, Full Time', 'Temporary', 'Temporary, Contract','Temporary, Full Time', 'Contract, Full Time, Flexi work','Temporary, Contract, Full Time','Freelance, Full Time, Flexi work']


# uniforming the labelling to either permanent full time or temporary full time

# df['employment_type'].map(lambda x: 'permanent_full_time' if x in permanent_full_time else 'temporary_full_time')
df=df[df.employment_type.isin(temporary_full_time+permanent_full_time)]

df['employment_type']=df['employment_type'].map(lambda x: 'permanent and fulltime' if x in permanent_full_time else 'temporary and fulltime')


## EDA and Cleaning Job Category

In [18]:
# df['job_category']=df['job_category'].map(lambda x: x.replace(' ',''))

In [19]:
# Top job category under the 'data search'
df.job_category.value_counts().sort_values(ascending=False)

Information Technology                                                                            520
Engineering                                                                                       127
Banking and Finance                                                                                90
Accounting / Auditing / Taxation                                                                   87
Admin / Secretarial                                                                                70
Sciences / Laboratory / R&D                                                                        65
Others                                                                                             63
Human Resources                                                                                    44
Advertising / Media                                                                                39
Logistics / Supply Chain                                                          

In [20]:
df.job_category.unique()

array(['Information Technology', 'Others',
       'Engineering, Information Technology', 'Banking and Finance',
       'Marketing / Public Relations ',
       'Education and Training, Sciences / Laboratory / R&D',
       'Environment / Health  , Healthcare / Pharmaceutical, Medical / Therapy Services',
       'Human Resources ', 'Advertising / Media ',
       'Environment / Health  , Healthcare / Pharmaceutical, Medical / Therapy Services, Sciences / Laboratory / R&D',
       'General Management', 'Sciences / Laboratory / R&D',
       'Consulting , Information Technology, Insurance', 'Legal',
       'Engineering', 'Information Technology, Public / Civil Service',
       'Design, Engineering, Information Technology',
       'Real Estate / Property Management', 'Admin / Secretarial',
       'Design, Engineering', 'Healthcare / Pharmaceutical',
       'Engineering, Information Technology, Logistics / Supply Chain',
       'Logistics / Supply Chain', 'Consulting ',
       'Accounting / Aud


### Pulling out job categories that are related to data analytics

#### 1) Information Technology (520)
#### 2) Engineering (127)
#### 3) Banking and Finance (90)
#### 4) Marketing / Public Relations (35)
#### 5) Advertising / Media (39)
#### 6) Sciences / Laboratory / R&D (65)
#### 7) Information Technology, Public / Civil Service (27)
#### 8) Banking and Finance, Information Technology (17)
#### 9) Consulting (31)
#### 10) Consulting , Information Technology (11)
#### 11) Information Technology, Insurance (1)
#### 12) Sales / Retail (32)
#### 13) General Management (16)
#### 14) Banking and Finance, Engineering, Information Technology, Telecommunications (1)
#### 15) Engineering, Information Technology, Professional Services (1)


In [21]:
relevant_jobs=['Information Technology','Engineering','Banking and Finance','Marketing / Public Relations ','Advertising / Media ','Sciences / Laboratory / R&D','Information Technology, Public / Civil Service',
                     'Banking and Finance, Information Technology','Consulting ','Consulting , Information Technology','Information Technology, Insurance','Sales / Retail ','General Management','Banking and Finance, Engineering, Information Technology, Telecommunications ',
                     'Engineering, Information Technology, Professional Services']
df=df[df['job_category'].isin(relevant_jobs)]

In [22]:
## cleaning the job category by removing punctuations

df['job_category']=df.job_category.map(lambda x: re.sub('[^a-zA-Z]',' ',x))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
df['job_category']=df['job_category'].map(lambda x: x.replace('r d','research and development'))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Changing all text to lowercase

In [24]:
df=df.applymap(lambda x: x.lower().strip() if isinstance(x,str) else x)

## EDA and Cleaning Postcode

In [25]:
# extracting the postcode for each job 
df['postcode']=df['job_address'].map(lambda x: 'NaN' if isinstance(x,str)!= True else x[-6:])

In [26]:
df['postcode'].value_counts().sort_values(ascending=False).head()

NaN       209
018981     31
117438     22
049909     22
486025     18
Name: postcode, dtype: int64

## EDA and Cleaning Company Name

In [27]:
# finding the company with the most job postings
df.company_name.value_counts().head()

company undisclosed                 102
a*star research entities             23
government technology agency         22
nanyang technological university     17
dbs bank ltd.                        16
Name: company_name, dtype: int64

In [28]:
# cleaning the company name. Removing PTE LTD,
df['company_name']=df['company_name'].map(lambda x: x.replace('pte. ltd.','').replace('pte ltd','').replace('ltd.','').replace('(pte.)','').replace('(singapore)','').replace('(singapore branch)','').replace('pte. limited','').replace('limited','').replace(' (s)',''))

In [29]:
df.company_name.head()

0    google asia pacific 
1             trakomatic 
3        intellect minds 
6               dbs bank 
7            niometrics  
Name: company_name, dtype: object

## EDA and Cleaning Job Requirements

In [30]:
for i in df.requirements.index:
    
    try:
        
        requirement=re.sub('[^a-zA-Z]',' ',df.requirements[i])
        requirement=requirement.replace('requirementsjob requirements','').replace('requirements','').replace('requirementsexperience','').replace('requirementsmandatory skills :','').replace('requirementsresponsibilities','').replace('requirementsminimum qualifications','').replace('requirementsrequirements','').replace('\xa0','').replace('requirementsqualifications','').split()
        ps=PorterStemmer()
        requirement=[ps.stem(word) for word in requirement if not word in stopwords.words('english')]
        requirement=' '.join(requirement)
        df['requirements'][i] = requirement
        
    except TypeError:
        df['requirements'][i] = 'NaN'
        
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [31]:
df.requirements

0       minimum qualif bachelor degre statist mathemat...
1       bachelor degre math comput scienc busi analyt ...
3       qualif advanc work sql knowledg experi work re...
6       master equival degre statist appli mathemat op...
7       valu bachelor higher degre comput scienc equiv...
8       valu ba bs ms equival experi appli data scienc...
10      degre bank financ busi legal document financi ...
11      degre inform technolog relev disciplin year re...
14      least two year relev work experi master phd de...
15      degre holder least year profession experi data...
17      role posit within etrad applic product team si...
19      diploma relev disciplin minimum year relat exp...
20      year experi enterpris architectureknowledg exp...
22      passion game strong mathemat skill digit marke...
23      skill knowledg mandatori indepth domain knowle...
24      degre comput scienc comput engin electron elec...
26      minimum qualif bachelor degre mathemat econom ...
27      least 

## EDA and Cleaning Job Description

In [32]:
# xf=[]
for i in df.description.index:
    
    try:
        
        description=re.sub('[^a-zA-Z]',' ',df.description[i])
        description=description.replace('roles   responsibilities','').replace('company overview','').split()
        ps=PorterStemmer()
        description=[ps.stem(word) for word in description if not word in stopwords.words('english')]
        description=' '.join(description)
#         xf.append(description)
        df['description'][i] = description
        
    except TypeError:
#         xf.append('NaN')
        df['description'][i] = 'NaN'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


## Question 1


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

### Bag of Words Model

#### Creating bag of words model for 

>1) Job Title

>2) Company Name

>3) Job Category

>4) Job Description

>5) Job Requirements

In [34]:
# # Get TFIDF for job title
# job_title_tvec = TfidfVectorizer(ngram_range=(1,3), stop_words='english', min_df=2, max_df=0.5, max_features=25)
# job_title_tvec.fit(df['company_name'])
# job_title_tvec_df = pd.DataFrame(job_title_tvec.transform(df['company_name']).todense(),
#                        columns=['title_[' + f + ']' for f in job_title_tvec.get_feature_names()])


In [35]:
# TFIDF on Company Name
company_name=df['company_name']
company_name_tvec=TfidfVectorizer(lowercase=True, ngram_range=(1,3),stop_words='english', max_df=0.5,min_df=2,max_features=25)
company_name_tvec.fit(company_name)

features=company_name_tvec.get_feature_names()
company_name_tvec.transform(company_name).todense()

company_name_df=pd.DataFrame(company_name_tvec.transform(company_name).todense(),columns=['Name_'+ n for n in features])
# company_name_tvec_df=pd.DataFrame(company_name.transform(company_name).todense(), columns=[features])
# company_name_df

In [36]:
company_name_df.head()

,Name_agency,Name_asia,Name_asia pacific,Name_bank,Name_company,Name_company undisclosed,Name_consulting,Name_entities,Name_global,Name_government,Name_international,Name_pacific,Name_private,Name_research,Name_research entities,Name_services,Name_singapore,Name_solutions,Name_star,Name_star research,Name_star research entities,Name_technologies,Name_technology,Name_undisclosed,Name_university
0,0.0,0.515509,0.607679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# TFIDF on Job Title
job_title=df['job_title']
job_title_tvec=TfidfVectorizer(lowercase=True, ngram_range=(1,3),stop_words='english', max_df=0.5,min_df=2,max_features=25)
job_title_tvec.fit(job_title)

job_title_df=pd.DataFrame(job_title_tvec.transform(job_title).todense(),columns=['Title_'+ n for n in job_title_tvec.get_feature_names()])


In [38]:
job_title_df.head()

,Title_analyst,Title_architect,Title_assistant,Title_associate,Title_business,Title_business analyst,Title_consultant,Title_data,Title_data scientist,Title_developer,Title_engineer,Title_executive,Title_lead,Title_management,Title_manager,Title_marketing,Title_project,Title_research,Title_scientist,Title_senior,Title_singapore,Title_software,Title_software engineer,Title_support,Title_technology
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.412948,0.00000,0.0,0.000000,0.0,0.63092,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.656821,0.0,0.0,0.0,0.0
1,0.755934,0.0,0.0,0.0,0.000000,0.0,0.0,0.654648,0.00000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.757957,0.00000,0.0,0.652305,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.482383,0.0,0.0,0.371983,0.57566,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.545482,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.757957,0.00000,0.0,0.652305,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [39]:
# TFIDF on Job Category
job_category=df['job_category']
job_category_tvec=TfidfVectorizer(lowercase=True, ngram_range=(1,3),stop_words='english', max_df=0.5,min_df=2,max_features=25)
job_category_tvec.fit(job_category)

job_category_df=pd.DataFrame(job_category_tvec.transform(job_category).todense(),columns=['Category_'+ n for n in job_category_tvec.get_feature_names()])


In [40]:
job_category_df.head()

,Category_advertising,Category_advertising media,Category_banking,Category_banking finance,Category_civil,Category_consulting,Category_engineering,Category_finance,Category_laboratory,Category_marketing,Category_marketing public,Category_marketing public relations,Category_media,Category_public,Category_public civil,Category_public civil service,Category_public relations,Category_relations,Category_retail,Category_sales,Category_sales retail,Category_sciences,Category_sciences laboratory,Category_service,Category_technology public civil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df.columns

Index(['company_name', 'job_address', 'job_title', 'job_category',
       'employment_type', 'salary_type', 'description', 'requirements',
       'salary_min', 'salary_max', 'salary_average', 'salary_high_low',
       'postcode'],
      dtype='object')

In [42]:
# TFIDF on Job Description
job_description=df['description']
job_description_tvec=TfidfVectorizer(lowercase=True, ngram_range=(1,3),stop_words='english', max_df=0.5,min_df=2,max_features=25)
job_description_tvec.fit(job_description)

job_description_df=pd.DataFrame(job_description_tvec.transform(job_description).todense(),columns=['Description_'+ n for n in job_description_tvec.get_feature_names()])



In [43]:
job_description_df.head()

,Description_client,Description_custom,Description_design,Description_engin,Description_ensur,Description_experi,Description_implement,Description_includ,Description_market,Description_new,Description_oper,Description_perform,Description_plan,Description_process,Description_product,Description_project,Description_provid,Description_report,Description_servic,Description_solut,Description_support,Description_technic,Description_technolog,Description_test,Description_use
0,0.111179,0.870423,0.042321,0.212210,0.046502,0.050471,0.047881,0.000000,0.169219,0.049553,0.046441,0.043798,0.000000,0.044237,0.133211,0.085991,0.040549,0.0,0.00000,0.324238,0.000000,0.050906,0.048669,0.00000,0.046685
1,0.312290,0.287639,0.000000,0.000000,0.000000,0.000000,0.000000,0.261924,0.316879,0.278378,0.260896,0.246046,0.000000,0.248513,0.000000,0.483080,0.000000,0.0,0.00000,0.260215,0.000000,0.285982,0.000000,0.00000,0.000000
2,0.568569,0.261844,0.216432,0.271311,0.000000,0.000000,0.000000,0.238435,0.000000,0.000000,0.237499,0.223981,0.000000,0.000000,0.227080,0.000000,0.207366,0.0,0.00000,0.000000,0.207366,0.260335,0.248894,0.00000,0.238749
3,0.000000,0.658312,0.108828,0.272846,0.000000,0.129783,0.000000,0.359676,0.145047,0.000000,0.000000,0.000000,0.000000,0.000000,0.114182,0.000000,0.000000,0.0,0.12413,0.476439,0.000000,0.000000,0.000000,0.00000,0.240099
4,0.000000,0.334854,0.000000,0.433701,0.076030,0.082519,0.156571,0.000000,0.000000,0.081018,0.000000,0.501259,0.084448,0.072326,0.000000,0.000000,0.000000,0.0,0.00000,0.151464,0.066296,0.000000,0.159146,0.36066,0.457979


In [44]:
# TFIDF on Job Requirements
job_requirements=df['requirements']
job_requirements_tvec=TfidfVectorizer(lowercase=True, ngram_range=(1,3),stop_words='english', max_df=0.5,min_df=2,max_features=25)
job_requirements_tvec.fit(job_requirements)

job_requirements_df=pd.DataFrame(job_requirements_tvec.transform(job_requirements).todense(),columns=['Requirements_'+ n for n in job_requirements_tvec.get_feature_names()])



In [45]:
job_requirements_df.head()

,Requirements_abil,Requirements_analyt,Requirements_applic,Requirements_busi,Requirements_comput,Requirements_data,Requirements_degre,Requirements_design,Requirements_develop,Requirements_engin,Requirements_environ,Requirements_excel,Requirements_good,Requirements_manag,Requirements_prefer,Requirements_process,Requirements_project,Requirements_relat,Requirements_scienc,Requirements_strong,Requirements_team,Requirements_technic,Requirements_technolog,Requirements_understand,Requirements_use
0,0.096900,0.114770,0.0,0.335368,0.226848,0.585792,0.179346,0.000000,0.000000,0.234067,0.000000,0.000000,0.000000,0.183872,0.111309,0.000000,0.000000,0.113591,0.466010,0.000000,0.307156,0.000000,0.000000,0.113093,0.000000
1,0.130550,0.309251,0.0,0.451829,0.305625,0.526145,0.120813,0.163939,0.000000,0.000000,0.163417,0.286411,0.000000,0.123862,0.000000,0.170239,0.159374,0.153037,0.156960,0.125971,0.000000,0.170526,0.000000,0.000000,0.000000
2,0.000000,0.075851,0.0,0.073881,0.074961,0.838817,0.059264,0.000000,0.000000,0.077347,0.080163,0.000000,0.000000,0.060760,0.000000,0.417548,0.000000,0.225214,0.076996,0.061794,0.135332,0.000000,0.000000,0.000000,0.082540
3,0.157076,0.372086,0.0,0.181211,0.000000,0.633049,0.145361,0.000000,0.000000,0.000000,0.000000,0.000000,0.340428,0.000000,0.180432,0.204828,0.000000,0.184131,0.000000,0.151566,0.165968,0.000000,0.180951,0.183325,0.202449
4,0.000000,0.000000,0.0,0.000000,0.485389,0.417808,0.383748,0.000000,0.440463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.498562,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
# Getting Training and Testing dataset
x_pred=pd.concat([company_name_df,job_title_df,job_category_df,job_description_df,job_requirements_df],axis=1)
y_true=df.salary_high_low.ravel()

In [47]:
x_train,x_test,y_train,y_test=train_test_split(x_pred,y_true,test_size=0.3,random_state=30)

In [48]:
ss=StandardScaler()
xs_train=ss.fit_transform(x_train)
xs_test=ss.fit_transform(x_test)

xs_train_df=pd.DataFrame(xs_train,columns=x_train.columns)
xs_test_df=pd.DataFrame(xs_test,columns=x_test.columns)

In [49]:
# Using Logistic Regression
lr=LogisticRegression()
lr.fit(x_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [50]:
pred = lr.predict(xs_test)
score = metrics.f1_score(y_test, pred)
print(classification_report(y_test, pred))
print('f1-score:', score)

              precision    recall  f1-score   support

           0       0.48      0.72      0.57       102
           1       0.81      0.61      0.69       202

   micro avg       0.64      0.64      0.64       304
   macro avg       0.64      0.66      0.63       304
weighted avg       0.70      0.64      0.65       304

f1-score: 0.6949152542372882


In [51]:
lr_coef = pd.DataFrame({'coef':lr.coef_.ravel(),
                    'mag':np.abs(lr.coef_.ravel()),
                    'pred':x_test.columns})

lr_coef.sort_values('mag', ascending=False, inplace=True)
lr_coef.head(10)

,coef,mag,pred
27,-2.751504,2.751504,Title_assistant
36,-1.970332,1.970332,Title_executive
3,1.473162,1.473162,Name_bank
116,1.204259,1.204259,Requirements_project
43,1.198753,1.198753,Title_scientist
123,1.195873,1.195873,Requirements_understand
13,1.152917,1.152917,Name_research
96,1.142771,1.142771,Description_technic
121,1.090553,1.090553,Requirements_technic
26,1.040408,1.040408,Title_architect


In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
# using gridsearch params

dtc_params = {
    'max_depth':[None,1,2,3,4],
    'max_features':[None,'log2','sqrt',2,3,4,5],
    'min_samples_split':[2,3,4,5,10,15,20,25,30,40,50]}

# set the gridsearch
dtc_gs=GridSearchCV(DecisionTreeClassifier(), dtc_params, cv=5,verbose=1)


In [54]:
dtc_gs.fit(xs_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 385 candidates, totalling 1925 fits


[Parallel(n_jobs=1)]: Done 1925 out of 1925 | elapsed:    4.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [None, 1, 2, 3, 4], 'max_features': [None, 'log2', 'sqrt', 2, 3, 4, 5], 'min_samples_split': [2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [55]:
# Best Estimator
dtc_best = dtc_gs.best_estimator_
print(dtc_gs.best_params_)
print(dtc_gs.best_score_)

{'max_depth': 4, 'max_features': None, 'min_samples_split': 2}
0.7005649717514124


In [56]:
fi = pd.DataFrame({
        'feature':x_train.columns,
        'importance':dtc_best.feature_importances_
    })

fi.sort_values('importance', ascending=False, inplace=True)
fi.head(10)

,feature,importance
27,Title_assistant,0.350077
36,Title_executive,0.202952
94,Description_solut,0.191162
35,Title_engineer,0.159562
122,Requirements_technolog,0.066787
101,Requirements_analyt,0.029459
67,Category_relations,0.000000
82,Description_includ,0.000000
66,Category_public relations,0.000000
91,Description_provid,0.000000


In [57]:
pred = dtc_best.predict(x_test)
score = metrics.f1_score(y_test, pred)
print(classification_report(y_test, pred, labels=[1,0], target_names=['high salary','low salary']))
print('f1_score=',score)

              precision    recall  f1-score   support

 high salary       0.66      1.00      0.80       202
  low salary       0.00      0.00      0.00       102

   micro avg       0.66      0.66      0.66       304
   macro avg       0.33      0.50      0.40       304
weighted avg       0.44      0.66      0.53       304

f1_score= 0.7984189723320159


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Summary

Based on the outputs from logistic regression and random forrest, it seems the logistic regression is output more decent and sensical outputs than random forrest even though random forrest is the better model. This can be seen from the classification report where the random forrest model is showing 100% recall for high salary which shouldnt be the case while the logistic regression is showing a balanced of recall and precision.

Both models are showing that the important features in determining the salary of a job is the job title

## Question 2

## EDA and Job Title Cleaning

In [58]:
for i in df.job_title.index:
    
        try:
        
            title=re.sub('[^a-zA-Z]',' ',df.job_title[i])
            df['job_title'][i] = title.split()
        
        except TypeError:
            df['job_title'][i] = 'NaN'


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [83]:
xf=pd.Series(index=df.job_title.index)

In [90]:
list_word=['data']

for index,sentence in enumerate(df.job_title):
    for word in sentence:
        if word in list_word:
            xf[index]=1

In [91]:
xf

0      1.0
1      1.0
3      1.0
6      1.0
7      1.0
8      1.0
10     1.0
11     NaN
14     NaN
15     NaN
17     NaN
19     NaN
20     NaN
22     NaN
23     NaN
24     NaN
26     NaN
27     NaN
30     NaN
31     NaN
32     NaN
34     NaN
35     NaN
36     NaN
37     NaN
38     NaN
41     NaN
43     NaN
45     NaN
48     NaN
      ... 
629    1.0
630    1.0
631    1.0
632    1.0
723    1.0
725    1.0
742    1.0
748    1.0
755    1.0
760    1.0
761    1.0
763    1.0
764    1.0
766    1.0
767    1.0
858    1.0
860    1.0
863    1.0
865    1.0
866    1.0
872    1.0
969    1.0
971    1.0
973    1.0
975    1.0
976    1.0
977    1.0
978    1.0
980    1.0
981    1.0
Length: 1097, dtype: float64

In [89]:
pd.concat([df,xf],axis=1)

,company_name,job_address,job_title,job_category,employment_type,salary_type,description,requirements,salary_min,salary_max,salary_average,salary_high_low,postcode,0
0,google asia pacific,"marina bay financial centre, 8 marina boulevar...","[data, science, lead, large, customer, sales, ...",information technology,permanent and fulltime,monthly,googl convent compani intend becom one true sh...,minimum qualif bachelor degre statist mathemat...,7700.0,15400.0,11550.0,1.0,018981,1.0
1,trakomatic,NaN,"[data, analyst]",information technology,permanent and fulltime,monthly,retail data analyst lead perform complex analy...,bachelor degre math comput scienc busi analyt ...,3500.0,4500.0,4000.0,0.0,NaN,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,intellect minds,"anson centre, 51 anson road 079904","[data, engineer]",information technology,permanent and fulltime,monthly,intellect mind singapor base compani sinc spec...,qualif advanc work sql knowledg experi work re...,5000.0,7000.0,6000.0,1.0,079904,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,dbs bank,"marina bay financial centre, 12 marina bouleva...","[avp, data, scientist, business, analytics, co...",information technology,permanent and fulltime,monthly,busi function lead bank asia db consum bank gr...,master equival degre statist appli mathemat op...,6500.0,11700.0,9100.0,1.0,018982,1.0
7,niometrics,"parkview square, 600 north bridge road 188778","[high, performance, data, engineer]",information technology,permanent and fulltime,monthly,invit part ambiti close knit team creat system...,valu bachelor higher degre comput scienc equiv...,5500.0,11000.0,8250.0,1.0,188778,1.0
8,niometrics,"parkview square, 600 north bridge road 188778","[data, scientist]",information technology,permanent and fulltime,monthly,niometr captur organis extract insight biggest...,valu ba bs ms equival experi appli data scienc...,5000.0,10000.0,7500.0,1.0,188778,1.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


#### Having issue with forming a target variable columns of 1 (Data Job Titles) and 0 (Non Data Job Title). 
#### After forming the target variable columns, I would create new predictors from the bag of words model created from job requirements and job description. Following the steps done for Question 1, I would create a new train and test split for predictors and target and form a logistic regression model and random forrest to determine the best model and the optimal features that would be used to determine what words in the job description and job requirements determines the difference between a data job title and a non data job title